We start by importing necessary libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

Then we read all the files

In [3]:
# Reading Boston datasets
Bos_cal = pd.read_csv("Boston calendar.csv")
Bos_lis = pd.read_csv("Boston listings.csv")
Bos_rev = pd.read_csv("Boston reviews.csv")

# Reading Seattle datasets
Sea_cal = pd.read_csv("Seattle calendar.csv")
Sea_lis = pd.read_csv("Seattle listings.csv")
Sea_rev = pd.read_csv("Seattle reviews.csv")

Now we want to know what the daatsets look like

In [4]:
Bos_cal.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [5]:
Bos_lis.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [6]:
Bos_rev.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


Now we want to check that the datasets share the same columns before we merge.

In [95]:
list(set(Bos_cal) - set(Sea_cal))

[]

In [96]:
list(set(Bos_rev) - set(Sea_rev))

[]

In [97]:
list(set(Bos_lis) - set(Sea_lis))

[]

As we see here, the Seattle listings dataset is missing 3 columns from the Boston's dataset, and I am not intrested in using these for solving any of my quistions I'll just drop them from Boston's dataset. 

In [56]:
Bos_lis.drop(columns=['interaction', 'access', 'house_rules'], inplace=True)

Then we make sure that the columns has the same types.

In [112]:
def checkTypes(df1, df2):
    for col in df1.columns:
        if df1[col].dtypes != df2[col].dtypes:
            print('{}, {} type: {}, {} type: {}'.
                  format(col, df1, df1[col].dtypes, df2, df2[col].dtypes))

In [67]:
# for col in Bos_lis.columns:
#     if Bos_lis[col].dtypes != Sea_lis[col].dtypes:
#         print('Column name: {}, Bos_lis type: {}, Sea_lis type: {}'.
#               format(col, Bos_lis[col].dtypes, Sea_lis[col].dtypes))

Column name: host_listings_count, Bos_lis type: int64, Sea_lis type: float64
Column name: host_total_listings_count, Bos_lis type: int64, Sea_lis type: float64
Column name: neighbourhood_group_cleansed, Bos_lis type: float64, Sea_lis type: object
Column name: has_availability, Bos_lis type: float64, Sea_lis type: object
Column name: jurisdiction_names, Bos_lis type: float64, Sea_lis type: object


In [117]:
print('cal datasets:', checkTypes(Bos_cal, Sea_cal), end='\n\n')
print('lis datasets:', checkTypes(Bos_lis, Sea_lis), end='\n\n')
print('rev datasets:', checkTypes(Bos_rev, Sea_rev))

cal datasets: None

lis datasets: None

rev datasets: None


We see that the lis datasets has columns with different types, so we will be changing one of them to match the other.

For the columns `host_listings_count` and `host_total_listings_count` we will change the `float` to `int` since it'll do no harm to the data. But `neighbourhood_group_cleansed`, `has_availability`, and `jurisdiction_names` we need more information first before we make a decision, so we will look how many data in each dataset for this column.

In [84]:
Bos_lis.neighbourhood_group_cleansed.notna().sum()

0

In [85]:
Sea_lis.neighbourhood_group_cleansed.notna().sum()

3818

In [86]:
Bos_lis.has_availability.notna().sum()

0

In [87]:
Sea_lis.has_availability.notna().sum()

3818

In [88]:
Bos_lis.jurisdiction_names.notna().sum()

0

In [89]:
Sea_lis.jurisdiction_names.notna().sum()

3818

Sweet, since the Boston's data for the the columns are NaN, we will change the Boston columns types to match Seattle's.

In [90]:
Bos_lis.host_listings_count = Bos_lis.host_listings_count.astype(float)
Bos_lis.host_total_listings_count = Bos_lis.host_total_listings_count.astype(float)
Bos_lis.neighbourhood_group_cleansed = Bos_lis.neighbourhood_group_cleansed.astype(object)
Bos_lis.has_availability = Bos_lis.has_availability.astype(object)
Bos_lis.jurisdiction_names = Bos_lis.jurisdiction_names.astype(object)

And now we merge

In [14]:
cal = pd.merge(Bos_cal, Sea_cal, how='outer', indicator=True)
d = {"left_only":"Boston", "right_only":"Seattle"}
cal["_merge"] = cal["_merge"].map(d)
cal.rename(columns={"_merge":"city"}, inplace=True)
cal.head()

,listing_id,date,available,price,city
0,12147973,2017-09-05,f,NaN,Boston
1,12147973,2017-09-04,f,NaN,Boston
2,12147973,2017-09-03,f,NaN,Boston
3,12147973,2017-09-02,f,NaN,Boston
4,12147973,2017-09-01,f,NaN,Boston


In [94]:
lis = pd.merge(Bos_lis, Sea_lis, how='outer')
lis.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [93]:
rev = pd.merge(Bos_rev, Sea_rev, how='outer')
rev.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...
